In [20]:
import pandas as pd

In [38]:
covidData = pd.read_csv("../../database/opendatasus/rn-opendata-2022.csv", on_bad_lines='skip')

In [39]:
#Visualizando Dados
covidData.head()
covidData.shape

(230764, 65)

# Verificando e Selecionando as Colunas

In [40]:
#Verificando a dimensão da Tabela
colunas = covidData.columns
colunas

Index(['source_id', 'dataNotificacao', 'dataInicioSintomas', 'sintomas',
       'profissionalSaude', 'racaCor', 'outrosSintomas', 'outrasCondicoes',
       'profissionalSeguranca', 'cbo', 'condicoes', 'sexo', 'estado',
       'estadoIBGE', 'municipio', 'municipioIBGE', 'origem',
       'estadoNotificacao', 'estadoNotificacaoIBGE', 'municipioNotificacao',
       'municipioNotificacaoIBGE', 'excluido', 'validado', 'dataEncerramento',
       'evolucaoCaso', 'classificacaoFinal', 'codigoEstrategiaCovid',
       'codigoBuscaAtivaAssintomatico', 'outroBuscaAtivaAssintomatico',
       'codigoTriagemPopulacaoEspecifica', 'outroTriagemPopulacaoEspecifica',
       'codigoLocalRealizacaoTestagem', 'outroLocalRealizacaoTestagem',
       'codigoRecebeuVacina', 'codigoDosesVacina', 'dataPrimeiraDose',
       'dataSegundaDose', 'codigoLaboratorioPrimeiraDose',
       'codigoLaboratorioSegundaDose', 'lotePrimeiraDose', 'loteSegundaDose',
       'codigoContemComunidadeTradicional', 'totalTestesRealizad

In [41]:
# Visualizando o formato das condições
covidData['classificacaoFinal'].value_counts()

Descartado                           5619
Confirmado Laboratorial              4485
Confirmado Clínico-Epidemiológico    1128
Confirmado por Critério Clínico       886
Síndrome Gripal Não Especificada      657
1                                       3
Name: classificacaoFinal, dtype: int64

In [42]:
# Separando as colunas mais importantes
colunasImportantes = ['source_id', 'sexo', 'racaCor', 'idade', 'sintomas', 'condicoes',
                      'dataPrimeiraDose', 'dataSegundaDose', 'classificacaoFinal', 'evolucaoCaso']
covidData = covidData[colunasImportantes]

# Calculando e Removendo os Valores Ausentes

In [43]:
#Verificando a quantidade de dados faltantes por colunas
from matplotlib import pyplot as plt

def visualizarQntFaltosos(covidData):
    faltantes = covidData.isnull().sum()
    faltantes_percentual = (covidData.isnull().sum() / len(covidData['source_id'])) * 100
    return faltantes_percentual
    
def plotarQntFaltosos(dados):
    nomes = dados.index
    valores = dados.values
    plt.barh(nomes, valores)
    plt.show

In [44]:
dados_faltosos = visualizarQntFaltosos(covidData)
dados_faltosos
# plotarQntFaltosos(dados_faltosos)

source_id              0.000000
sexo                   0.000433
racaCor                0.002600
idade                  0.363575
sintomas               0.000000
condicoes             89.077152
dataPrimeiraDose      31.541315
dataSegundaDose       35.425370
classificacaoFinal    94.462741
evolucaoCaso          94.441941
dtype: float64

# Limpeza de Dados - Classificação Final

```Descartado, Confirmado [Laboratorial, Clínico-Epidemiológico, Critério Clínico] e  Síndrome Gripal Não Especificada```

In [45]:
# Visualizando o formato das condições
covidData['classificacaoFinal'].value_counts()

Descartado                           5619
Confirmado Laboratorial              4485
Confirmado Clínico-Epidemiológico    1128
Confirmado por Critério Clínico       886
Síndrome Gripal Não Especificada      657
1                                       3
Name: classificacaoFinal, dtype: int64

#### i. Remoção dos valores ausentes

In [46]:
classificacao = 'classificacaoFinal'
atributosVazios = covidData.loc[covidData[classificacao].isnull()]
covidData = covidData.drop(atributosVazios.index)

#### ii. Juntando os valores Confirmados

In [47]:
covidData.loc[covidData[classificacao].str.contains(pat = 'Confirmado'), [classificacao]] = 'Confirmado';
covidData[classificacao].value_counts()

Confirmado                          6499
Descartado                          5619
Síndrome Gripal Não Especificada     657
1                                      3
Name: classificacaoFinal, dtype: int64

In [48]:
covidData[classificacao].value_counts()

Confirmado                          6499
Descartado                          5619
Síndrome Gripal Não Especificada     657
1                                      3
Name: classificacaoFinal, dtype: int64

#### iii. Removendo os valores diferentes de Confirmados

In [49]:
dadosNaoConfirmados = covidData.loc[((covidData[classificacao] != 'Confirmado'))]

covidData = covidData.drop(dadosNaoConfirmados.index)

In [50]:
covidData[classificacao].value_counts()

Confirmado    6499
Name: classificacaoFinal, dtype: int64

In [51]:
dados_faltosos = visualizarQntFaltosos(covidData)
dados_faltosos

source_id              0.000000
sexo                   0.000000
racaCor                0.015387
idade                  0.000000
sintomas               0.000000
condicoes             93.537467
dataPrimeiraDose      35.190029
dataSegundaDose       37.913525
classificacaoFinal     0.000000
evolucaoCaso           2.800431
dtype: float64

# Limpeza de Dados - Atibuto Evolução

``` Pode conter (Cancelado, Cura, Ignorado, Em tratamento domiciliar, Óbito, Internado, Internado em UTI ```

In [52]:
alvo = 'evolucaoCaso'
#covidData[alvo].unique()
covidData[alvo].value_counts()

Cura                        5609
Em tratamento domiciliar     490
Ignorado                     126
Óbito                         56
Cancelado                     31
Internado                      4
Internado em UTI               1
Name: evolucaoCaso, dtype: int64

#### i. Remoção dos valores ausentes

In [53]:
atributosVazios = covidData.loc[covidData[alvo].isnull()]
covidData = covidData.drop(atributosVazios.index)

In [54]:
# visualizarQntFaltosos(covidData)
covidData[alvo].value_counts()

Cura                        5609
Em tratamento domiciliar     490
Ignorado                     126
Óbito                         56
Cancelado                     31
Internado                      4
Internado em UTI               1
Name: evolucaoCaso, dtype: int64

#### ii. Removendo os valores Ignorado e Cancelado

In [55]:
dadosIgnoradosCancelados = covidData.loc[((covidData[alvo] == 'Ignorado') | 
                                          (covidData[alvo] == 'Cancelado'))]

covidData = covidData.drop(dadosIgnoradosCancelados.index)

In [56]:
covidData[alvo].value_counts()

Cura                        5609
Em tratamento domiciliar     490
Óbito                         56
Internado                      4
Internado em UTI               1
Name: evolucaoCaso, dtype: int64

#### iii. Agrupando os valores de 'Internado'

In [58]:
covidData.loc[covidData[alvo] == 'Internado em UTI', [alvo]] = 'Internado';
covidData[alvo].value_counts()

Cura                        5609
Em tratamento domiciliar     490
Óbito                         56
Internado                      5
Name: evolucaoCaso, dtype: int64

#### iv. Agrupando os valores de 'Tratamento Domiciliar'

In [59]:
covidData.loc[covidData[alvo] == 'Em tratamento domiciliar', [alvo]] = 'Cura';
covidData[alvo].value_counts()

Cura         6099
Óbito          56
Internado       5
Name: evolucaoCaso, dtype: int64

In [60]:
dados_faltosos = visualizarQntFaltosos(covidData)
dados_faltosos

source_id              0.000000
sexo                   0.000000
racaCor                0.016234
idade                  0.000000
sintomas               0.000000
condicoes             93.620130
dataPrimeiraDose      34.837662
dataSegundaDose       37.581169
classificacaoFinal     0.000000
evolucaoCaso           0.000000
dtype: float64

In [61]:
covidData.shape

(6160, 10)

# Coletando os Sintomas

``` Pode conter - Assintomático, Febre, Dor de Garganta, Dispneia, Tosse,  Coriza, Dor de Cabeça, Distúrbios gustatórios, Distúrbios olfativos ```

In [62]:
covidData.head()

,source_id,sexo,racaCor,idade,sintomas,condicoes,dataPrimeiraDose,dataSegundaDose,classificacaoFinal,evolucaoCaso
273,eetgqKPruw,Masculino,Preta,39.0,"Coriza, Dor de Cabeça, Tosse, Febre, Dispneia,...",NaN,2021-07-01,2021-10-13,Confirmado,Cura
289,aOl819OygW,Masculino,Ignorado,48.0,Outros,NaN,2021-07-01,NaN,Confirmado,Cura
299,y0XUQvdgIV,Masculino,Parda,42.0,"Coriza, Dor de Cabeça, Tosse, Dor de Garganta",NaN,2021-07-01,NaN,Confirmado,Cura
301,3KwWZV0ODQ,Feminino,Preta,46.0,"Coriza, Dor de Cabeça, Tosse, Febre, Dor de Ga...",NaN,2021-07-01,2021-07-31,Confirmado,Cura
346,EeGqCf9XLd,Feminino,Parda,27.0,"Tosse, Febre, Dor de Garganta",NaN,2021-01-20,2021-10-18,Confirmado,Cura


In [69]:
sintomas = {"assintomatico": 'Assintomático', 
            "febre": 'Febre', 
            "dorDeGarganta": 'Dor de Garganta',
            "dispneia": 'Dispneia',
            "tosse": 'Tosse',
            "coriza": 'Coriza',
            "dorDeCabeca": 'Dor de Cabeça',
            "disturbiosGustatórios": 'Distúrbios gustatórios',
            "disturbiosOlfativos": 'Distúrbios olfativos'}

In [72]:
for chave in sintomas.keys():
    covidData[chave] = covidData['sintomas'].apply(lambda x: 1 if sintomas[chave] in x else 0)

In [73]:
covidData.head()
# 'Febre' in covidData['sintomas'] 

,source_id,sexo,racaCor,idade,sintomas,condicoes,dataPrimeiraDose,dataSegundaDose,classificacaoFinal,evolucaoCaso,febre,assintomatico,dorDeGarganta,dispneia,tosse,coriza,dorDeCabeca,disturbiosGustatórios,disturbiosOlfativos
273,eetgqKPruw,Masculino,Preta,39.0,"Coriza, Dor de Cabeça, Tosse, Febre, Dispneia,...",NaN,2021-07-01,2021-10-13,Confirmado,Cura,1,0,1,1,1,1,1,0,0
289,aOl819OygW,Masculino,Ignorado,48.0,Outros,NaN,2021-07-01,NaN,Confirmado,Cura,0,0,0,0,0,0,0,0,0
299,y0XUQvdgIV,Masculino,Parda,42.0,"Coriza, Dor de Cabeça, Tosse, Dor de Garganta",NaN,2021-07-01,NaN,Confirmado,Cura,0,0,1,0,1,1,1,0,0
301,3KwWZV0ODQ,Feminino,Preta,46.0,"Coriza, Dor de Cabeça, Tosse, Febre, Dor de Ga...",NaN,2021-07-01,2021-07-31,Confirmado,Cura,1,0,1,0,1,1,1,0,0
346,EeGqCf9XLd,Feminino,Parda,27.0,"Tosse, Febre, Dor de Garganta",NaN,2021-01-20,2021-10-18,Confirmado,Cura,1,0,1,0,1,0,0,0,0


# Organizando as Comorbidades
#### 

In [67]:
# Comorbidades
comorbidades = {"diabetes": 'Diabetes', 
                "obesidade": 'Obesidade', 
                "renal": 'Doenças renais crônicas em estágio avançado',
                "respiratoria": 'Doenças respiratórias crônicas descompensadas',
                "imunossupressao": 'Imunossupressão',
                "fragilidadeImuno": 'Portador de doenças cromossômicas ou estado de fragilidade imunológica',
                "gestante": 'Gestante',
                "cardiaca": 'Doenças cardíacas crônicas',
                "puerpera": 'Puérpera'}

x = comorbidades.keys()


# stringX = 'Doenças cardíacas crônicas, Diabetes'
# comorbidades['diabetes'] in stringX

dict_values(['Diabetes', 'Obesidade', 'Doenças renais crônicas em estágio avançado', 'Doenças respiratórias crônicas descompensadas', 'Imunossupressão', 'Portador de doenças cromossômicas ou estado de fragilidade imunológica', 'Gestante', 'Doenças cardíacas crônicas', 'Puérpera'])

In [ ]:
for chave in comorbidades.keys():
    print(chave)

In [45]:
# Substituindo o valor das comorbidades
covidNewData.loc[covidNewData.FATOR_RISC == 2, ["PUERPERA", "CARDIOPATI", "HEMATOLOGI",
            "SIND_DOWN", "HEPATICA", "ASMA","DIABETES", "NEUROLOGIC", "PNEUMOPATI",
            "IMUNODEPRE", "RENAL", "OBESIDADE"]] = 2;
covidNewData.RENAL.value_counts()

/home/wallece/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


2.0    424
1.0     53
Name: RENAL, dtype: int64

In [46]:
covidNewData.shape

(1120, 165)

In [47]:
newCovidData = covidNewData.drop(covidNewData[covidNewData.CLASSI_FIN != 5].index, inplace = False)
#covidNewData.shape
newCovidData.shape

(762, 165)

In [48]:
#Filtrando os atributos
attributes = ["CS_SEXO", "NU_IDADE_N", "CS_RACA", "SURTO_SG", "FEBRE", "TOSSE", "GARGANTA", 
              "DISPNEIA", "DESC_RESP", "SATURACAO", "DIARREIA","VOMITO", "FATOR_RISC", "PUERPERA", "CARDIOPATI", 
              "HEMATOLOGI", "SIND_DOWN", "HEPATICA", "ASMA", "DIABETES", "NEUROLOGIC","PNEUMOPATI",
              "IMUNODEPRE", "RENAL", "OBESIDADE", "DT_INTERNA", "DT_EVOLUCA", "CLASSI_FIN", "SUPORT_VEN", "UTI"]
# 'EVOLUCAO'

dbCovid = newCovidData[attributes];
dbCovid.shape

(762, 30)

In [49]:
write = pd.ExcelWriter('uti_dados_pp_Jan2021.xlsx')
dbCovid.to_excel(write, 'dados', index = False)
write.save()

In [47]:
# Lista de Comorbidades
"""
Diabetes
Obesidade
Doenças renais crônicas em estágio avançado (graus 3, 4 e 5)
Doenças respiratórias crônicas descompensadas 
Imunossupressão
Portador de doenças cromossômicas ou estado de fragilidade imunológica
Gestante
Doenças cardíacas crônicas 
Puérpera (até 45 dias do parto)
"""

'\nDiabetes\nObesidade\nDoenças renais crônicas em estágio avançado (graus 3, 4 e 5)\nDoenças respiratórias crônicas descompensadas \nImunossupressão\nPortador de doenças cromossômicas ou estado de fragilidade imunológica\nGestante\nDoenças cardíacas crônicas \nPuérpera (até 45 dias do parto)\n'